<img src="../img/python-logo-no-text.png"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>Workshop: Multiprocessing</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<!-- <div style="text-align:center;">workshops/workshop_420_multiprocessing</div> -->




# Monte-Carlo Methods

Monte Carlo methods are statistical algorithms that use repeated calculations on
random numbers to get numerical results. In this exercise we want to calculate the
number $\pi$ using Monte Carlo methods. The basic idea is as follows:

- $\pi$ corresponds to the area of a circle with radius 1.
- We generate two random numbers x and y, each lying between 0 and 1.
- We calculate x^2 + y^2 and test if the value \leq is 1. In this case, the point
  $(x,y)$ is contained in the circle of radius 1 around the origin, otherwise this is
  not the case.
- We do this calculation very often, counting the percentage of points in the circle.
- Since all points are in the first quadrant, we get anapproximation of $\pi$ by
  multiplying the result by 4.


The following function creates a point $(x, y)$ as described:

In [ ]:
from random import random


def get_random_point():
    return random(), random()


Implement a function `is_in_circle(x, y) -> bool` that checks whether such a point is
inside a circle with radius 1.

In [ ]:
def is_in_circle(x, y) -> bool:
    return x**2 + y**2 <= 1


Implement a function `is_random_point_in_circle() -> bool` that checks if a randomly
chosen point is inside the circle with radius 1.

In [ ]:
def is_random_point_in_circle() -> bool:
    return is_in_circle(*get_random_point())


How can you apply the method described above to compute $\pi$? Implement a sequential
version using `is_random_point_in_circle()` and at least one parallel version. Test
the performance of different approaches.

In [ ]:
def compute_pi_sequentially(num_iterations):
    result = []
    for _ in range(num_iterations):
        result.append(is_random_point_in_circle())
    return 4 * sum(result) / len(result)

In [ ]:
NUM_ITERATIONS = 10_000_000

In [ ]:
if __name__ == "__main__":
    print("Sequential value:")
    print(compute_pi_sequentially(NUM_ITERATIONS))

In [ ]:
from timeit import timeit

if __name__ == "__main__":
    print("Sequential:")
    print(timeit(lambda: compute_pi_sequentially(NUM_ITERATIONS), number=5))

In [ ]:
def compute_pi_sequentially_2(num_iterations):
    num_points_in_circle = 0
    total_points = 0
    for _ in range(num_iterations):
        total_points += 1
        is_in_circle = is_random_point_in_circle()
        if is_in_circle:
            num_points_in_circle += 1
    return 4 * num_points_in_circle / total_points

In [ ]:
if __name__ == "__main__":
    print("Better sequential value:")
    print(compute_pi_sequentially_2(NUM_ITERATIONS))

In [ ]:
if __name__ == "__main__":
    print("Better sequential:")
    print(timeit(lambda: compute_pi_sequentially_2(NUM_ITERATIONS), number=5))

In [ ]:
from multiprocessing import Pool


def bad_parallel_version(num_iterations):
    points = (get_random_point() for _ in range(num_iterations))
    with Pool(processes=16) as pool:
        result = list(pool.starmap(is_in_circle, points))
    return 4 * sum(result) / len(result)

In [ ]:
if __name__ == "__main__":
    print("Bad parallel value:")
    print(bad_parallel_version(NUM_ITERATIONS))

In [ ]:
if __name__ == "__main__":
    print("Bad parallel (16 processes):")
    print(timeit(lambda: bad_parallel_version(NUM_ITERATIONS), number=5))

In [ ]:
def better_parallel_version(num_iterations, num_processes=16):
    iterations_per_process = [num_iterations // num_processes] * num_processes
    with Pool(processes=num_processes) as pool:
        result = list(pool.imap(compute_pi_sequentially_2, iterations_per_process))
    return sum(result) / len(result)

In [ ]:
if __name__ == "__main__":
    print("Better parallel value:")
    print(better_parallel_version(NUM_ITERATIONS))

In [ ]:
if __name__ == "__main__":
    print("Better parallel (8 processes):")
    print(timeit(lambda: better_parallel_version(NUM_ITERATIONS, 8), number=5))

In [ ]:
if __name__ == "__main__":
    print("Better parallel (16 processes):")
    print(timeit(lambda: better_parallel_version(NUM_ITERATIONS, 16), number=5))

In [ ]:
if __name__ == "__main__":
    print("Better parallel (32 processes):")
    print(timeit(lambda: better_parallel_version(NUM_ITERATIONS, 32), number=5))